## AI Gallery

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import torch
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import faiss  # FAISS for fast similarity search
from tqdm import tqdm

In [ ]:
# Load CLIP Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Paths
folder_path = r"{Images Folder}"
description_store = {}
vector_store = {}

# Path to save/load existing vector store
vector_store_path = "vector_store.npy"
faiss_index_path = "faiss_index.index"

c:\Users\2103s\miniconda3\envs\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Check if vector store already exists
if os.path.exists(vector_store_path):
    vector_store = np.load(vector_store_path, allow_pickle=True).item()
else:
    vector_store = {}

In [4]:
image_features = []

for img_file in tqdm(os.listdir(folder_path)):
    if img_file.endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(folder_path, img_file)
        
        # Check if image feature is already generated
        if img_path in vector_store:
            continue  # Skip if already processed
        
        image = Image.open(img_path).convert("RGB")
        
        # Generate Image Features
        inputs = processor(images=image, return_tensors="pt").to(device)
        outputs = model.get_image_features(**inputs)
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)  # Normalize to unit length
        
        # Store Image Embeddings with Folder Information
        vector_store[img_path] = outputs.detach().cpu().numpy()
        image_features.append(outputs.detach().cpu().numpy())

100%|██████████| 762/762 [00:00<00:00, 753787.65it/s]

In [ ]:
# Convert to NumPy for FAISS
image_features = np.vstack(image_features)

# Save vector store
np.save("vector_store.npy", vector_store)  # Save embeddings with folder details

if os.path.exists(faiss_index_path):
    faiss_index = faiss.read_index(faiss_index_path)
    faiss_index.add(image_features)
else:
    dimension = image_features.shape[1]  # Reduced feature dimension
    faiss_index = faiss.IndexFlatIP(dimension)  # Use inner product (IP) for cosine similarity
    faiss_index.add(image_features)  # Add vectors to the index

# Save FAISS index to file
faiss.write_index(faiss_index, faiss_index_path)

In [ ]:
# Query
query = "three persons"
query_inputs = processor(text=[query], return_tensors="pt").to(device)
query_vector = model.get_text_features(**query_inputs)
query_vector = query_vector / query_vector.norm(p=2, dim=-1, keepdim=True)  # Normalize to unit length

# Perform top-k search using FAISS
top_k = 5
distances, top_k_indices = faiss_index.search(query_vector.cpu().detach().numpy(), top_k)

# Retrieve top-k results (excluding cluster information)
top_k_results = [
    {"image_path": list(vector_store.keys())[i], "distance": distances[0][j]}
    for j, i in enumerate(top_k_indices[0])
]

# Output results
for result in top_k_results:
    print(f"Image Path: {result['image_path']}")
    print(f"Distance: {result['distance']}")